In [3]:
import geopandas as gpd

#This is critical for ensuring that the ATL segments match one another
atl_raster_ymins = []
for i in range(0, 8):
    b1 = i * 4
    b2 = b1 + 4
    gdf = gpd.read_file(f'../make_image/atl 1C/atl json/atl_{b1}_{b2}.json')
    xmin, ymin, xmax, ymax = gdf.total_bounds
    atl_raster_ymins.append(ymin)

In [42]:
import math
import numpy as np
import pandas as pd
import geopandas as gpd

min_loc_val = [(35, 545),
 (46, 358),
 (66, 171),
 (84, 327),
 (147, 490),
 (206, 519),
 (80, 549),
 (305, 940)]

step = -0.18
ROT_C = np.arange(17.88, 17.88 + 8 * step, step)

def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def get_offset(i, min_loc_val, ANGLE):
    atl_df = pd.read_csv('unshifted_gt3r_full.csv')
    b1 = i * 4
    b2 = b1 + 4
    chirop_df = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')
    x_min = min(chirop_df['x'])
    x_max = max(chirop_df['x'])
    y_min = min(chirop_df['y'])
    y_max = max(chirop_df['y'])
    ATL_max_y = atl_raster_ymins[i] + 4000
    ATL_min_y = ATL_max_y - 2000

    atl_df = atl_df[(atl_df['y'] >= ATL_min_y) & (atl_df['y'] <= ATL_max_y)]
    TL_POINT = get_chirop_tl_coord(i, ANGLE)

    #revert ATL to Chirop TL
    x_max = max(atl_df['x'])
    y_max = max(atl_df['y'])

    offset = (TL_POINT[0] - x_max, TL_POINT[1] - y_max)
    atl_df['x'] = atl_df['x'] + offset[0]
    atl_df['y'] = atl_df['y'] + offset[1]

    atl_df.to_csv(f'debug_shifted_atl_{b1}_{b2}.csv', index=False)
    #At this point, the ATL has been matched to the top-left corner.
    #We now want to shift with the best fit (in original coordinates).
    
    (best_x, best_y) = min_loc_val[i]
    best_fit = (best_x, -best_y)
    print(best_fit)
    best_fit = rotate((0,0), best_fit, -np.deg2rad(ANGLE))
    print(f'total shift for {b1}-{b2}: {(best_fit[0] + offset[0], best_fit[1] + offset[1])}')
    atl_df['x'] = atl_df['x'] + best_fit[0]
    atl_df['y'] = atl_df['y'] + best_fit[1]
    atl_df.to_csv(f'debug_shifted_atl_{b1}_{b2}.csv', index=False)

for i in range(0,8):
    get_offset(i, min_loc_val, 17.88)

(570762.3815780896, 9497300.033293018)
(35, -545)
total shift for 0-4: (-13.983128056248404, -37.76168354328877)
(572066.3416800046, 9501348.838212125)
(46, -358)
total shift for 4-8: (-4.156852066885122, -41.30669591305548)
(573272.7012045436, 9505230.609904572)
(66, -171)
total shift for 8-12: (-26.158582361057327, -26.351194070805434)
(574496.8261999603, 9509134.984480413)
(84, -327)
total shift for 12-16: (-44.817967097552526, -18.554778888084456)
(575733.9324536205, 9513079.289737793)
(147, -490)
total shift for 16-20: (-25.714324206896578, -37.2455571082254)
(576926.2194879231, 9516919.30801871)
(206, -519)
total shift for 20-24: (-14.386710081885667, -37.81166511578533)
(578198.444107707, 9521073.96961165)
(80, -549)
total shift for 24-28: (-59.99804257028886, 436.78038105349174)
(579373.1729606858, 9524984.499241997)
(305, -940)
total shift for 28-32: (-58.044521654054165, -36.51028702204394)


In [8]:
get_offset(0, min_loc_val, ROT_C[0])

(570762.3815780896, 9497300.033293018)
(-90.78956921978838, 388.9090563626479)
